# 决策树

- ID3（基于信息增益）
- C4.5（基于信息增益比）
- CART（gini指数）

#### entropy：$H(x) = -\sum_{i=1}^{n}p_i\log{p_i}$

#### conditional entropy: $H(X|Y)=\sum{P(X|Y)}\log{P(X|Y)}$

#### information gain : $g(D, A)=H(D)-H(D|A)$

#### information gain ratio: $g_R(D, A) = \frac{g(D,A)}{H(A)}$

#### gini index:$Gini(D)=\sum_{k=1}^{K}p_k\log{p_k}=1-\sum_{k=1}^{K}p_k^2$

其中特征A对数据集D的经验条件熵$H(D|A)$

$$H(D|A)=\sum_{i=1}^{n}\frac{|D_i|}{|D|}H(D_i)\sum_{k=1}^{K}\log{\frac{|D_{ik}|}{|D_i|}}$$

$D_i$表示根据特征A的取值将D划分为n个子集$D_1,D_2,...,D_n$

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math
from math import log

import pprint

In [ ]:
# 书上例5.1
def create_data():
    datasets = [['青年', '否', '否', '一般', '否'],
               ['青年', '否', '否', '好', '否'],
               ['青年', '是', '否', '好', '是'],
               ['青年', '是', '是', '一般', '是'],
               ['青年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '一般', '否'],
               ['中年', '否', '否', '好', '否'],
               ['中年', '是', '是', '好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['中年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '非常好', '是'],
               ['老年', '否', '是', '好', '是'],
               ['老年', '是', '否', '好', '是'],
               ['老年', '是', '否', '非常好', '是'],
               ['老年', '否', '否', '一般', '否'],
               ]
    labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']
    # 返回数据集和每个维度的名称
    return datasets, labels

In [ ]:
datasets, labels = create_data()

In [ ]:
train_data = pd.DataFrame(datasets, columns=labels)

In [ ]:
train_data

In [ ]:
# 熵 H(D)
def cal_ent(datasets):
    n = len(datasets)
    label_count = {}
    for i in range(n):
        label = datasets[i][-1]
        if label in label_count:
            label_count[label] +=1
        else:
            label_count[label] = 1
    ent = - sum( [(c/n * log(c/n,2)) for c in label_count.values()] ) 
    return ent

# 经验条件熵 H(D|A) 第axis个特征
def cal_cond_ent(datasets ,axis=0):
    n = len(datasets)
    feature_count = {}
    ent = 0
    for i in range(n):
        if datasets[i][axis] in feature_count:
            feature_count[datasets[i][axis]] +=1
        else:
            feature_count[datasets[i][axis]] = 1
    for d in feature_count:
        ent += feature_count[d]/n * cal_ent([p for p in datasets if p[axis] == d]) 
    return ent

# 信息增益
def info_gain(datasets):
    features = datasets.shape[1] -1
    info_gains = []
    ent = cal_ent(datasets)
    for i in range(features):
        info_gains.append((i, ent - cal_cond_ent(datasets,i)))
    for i in range(features):
        print('特征:{0} info_gain = {1:.3f}'.format(labels[i],info_gains[i][-1]))
    
    best = max(info_gains, key= lambda x:x[-1])
    print('最优特征:{}'.format(labels[best[0]]))
    return info_gains

In [ ]:
data = np.array(train_data)
info_gain(data)

# ID3

In [ ]:
# 定义节点类 二叉树
class Node:
    def __init__(self, leaf=True, label=None,feature=None):
        self.leaf = leaf       # 是否为叶子节点
        self.label = label     # 叶节点的label；若是内部节点，则为none
        self.feature = feature # 该节点的特征Ag
        self.tree = {}         # Ag每个可能的值构成
        self.result = {'label:': self.label, 'feature': self.feature, 'tree': self.tree}

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node #添加特征Ag取值为val的节点

    def predict(self, features):
        if self.leaf is True:
            return self.label
        return self.tree[features[self.feature]].predict(features) # 访问子树, features[self.feature]表示测试用例在该节点特征的取值

In [ ]:
class ID3_DTree:
    def __init__(self,epsilon=0.1):
        self.tree = {}
        self.epsilon = epsilon
    
    def __repr__(self):
        return '{}'.format(self.result)

    # 熵 H(D)
    def cal_ent(self, datasets):
        n = len(datasets)
        label_count = {}
        for i in range(n):
            label = datasets[i][-1]
            if label in label_count:
                label_count[label] +=1
            else:
                label_count[label] = 1
        ent = - sum( [(c/n * log(c/n,2)) for c in label_count.values()] ) 
        return ent
    
    # 经验条件熵 H(D|A) 第axis个特征
    def cal_cond_ent(self, datasets ,axis=0):
        n = len(datasets)
        feature_count = {}
        cond_ent = 0
        for i in range(n):
            if datasets[i][axis] in feature_count:
                feature_count[datasets[i][axis]] +=1
            else:
                feature_count[datasets[i][axis]] = 1
        for d in feature_count:
            cond_ent += feature_count[d]/n * cal_ent([p for p in datasets if p[axis] == d]) 
        return cond_ent
    
    def info_gain(self, ent, con_ent):
        return ent - con_ent
    
    # 信息增益
    def cal_info_gain(self, datasets):
        features = len(datasets[0]) -1
        info_gains = []
        ent = cal_ent(datasets)
        for i in range(features):
            info_gains.append((i, ent - cal_cond_ent(datasets,axis=i))) #第i个特征
        
        best = max(info_gains, key= lambda x:x[-1])
        return best #（axis， info_gain)
    
    def train(self, datasets,features):
        y_train = datasets[:,-1]
        y_set = set(y_train)
        
        # (1)如果所有实例属于同一类别Ck
        if len(y_set) == 1:
            return Node(leaf=True, label=y_set.pop(), feature=None) 

        # (2)如果特征集合为空
        label_count = {}
        for i in y_train:
            if i not in label_count:
                label_count[i] = 1
            else:
                label_count[i] +=1
        max_class = max(label_count, key=lambda x:x[-1])[0] # 实例最多的类

        if len(features) == 0:
            return Node(leaf=True, label=max_class)
        
        # (3)计算信息增益，选择信息增益最大特征Ag
        max_feature, max_gain = self.cal_info_gain(datasets)
        
        # (4)如果Ag的信息增益小于阈值eta,则置T为单节点树，并将D中是实例数最大的类Ck作为该节点的类标记，返回T
        if max_gain < self.epsilon:
            return Node(leaf=True, label=max_class)
        
        # (5)构建树
        node_tree = Node(leaf=False,feature=max_feature)
        sub_features = list(filter(lambda x:x!=max_feature, features)) # 剩余特征集 {A-Ag}
        max_feature_data = datasets[:,max_feature]                      # 信息增益最大的特征对应列
        feature_value_list = list(set(max_feature_data))                # 保存信息增益最大的特征可能的取值 
        
        # 根据Ag=ai将D划分为子集Di
        for f in feature_value_list:
            index = []
            for i in range(len(datasets)):
                if datasets[i][max_feature] == f: 
                    index.append(i) # Di所在的行
            sub_data = datasets[index]
            sub_tree = self.train(sub_data,features) # 递归生成子树
            node_tree.add_node(f, sub_tree)        
        
        return node_tree
                   
              
    def fit(self,datasets):
        self.tree = self.train(datasets,list(range(len(datasets[0])-1))) # feature为特征序号列表
        return self.tree
    
    def predict(self, X_test):
        return self.tree.predict(X_test)
    
    

In [ ]:
data = np.array(train_data)
dt = ID3_DTree()
tree = dt.fit(data)

In [ ]:
tree

In [ ]:
labels = [u'年龄', u'有工作', u'有自己的房子', u'信贷情况', u'类别']

In [ ]:
dt.predict(['老年', '否', '否', '一般'])